In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
dfCOVID = pd.read_csv('DOH COVID Data Drop_ 20220917 - 04 Case Information_batch_3.csv')

C:\Users\baja0\AppData\Local\Temp\ipykernel_11232\441803758.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCOVID = pd.read_csv('DOH COVID Data Drop_ 20220917 - 04 Case Information_batch_3.csv')


In [4]:
df_active_cases = dfCOVID[(dfCOVID.HealthStatus != 'RECOVERED') & (dfCOVID.HealthStatus != 'DIED')].groupby('CityMuniPSGC', as_index=False).agg(active_cases = ('CityMuniPSGC', 'count')).sort_values('active_cases', ascending=False)
df_active_cases

,CityMuniPSGC,active_cases
1047,PH137404000,2366
1046,PH137403000,856
1056,PH137604000,726
1059,PH137607000,682
1054,PH137602000,674
...,...,...
801,PH086004000,1
798,PH084822000,1
796,PH084814000,1
794,PH084811000,1


In [5]:
dfHW = pd.read_excel('200108-2018-doh-human-resources-geographical-distribution-profession-and-by-facility-ownership-xlsx-1.xlsx')
dfHW.head()

,Region,Region code,Province,Province code,City/Municipality,City/Municipality code,Classification,Dentist,Doctor: Clinical,Medical Technologist,Midwife,Nurse,Nutritionist/ Dietician,Occupational Therapist,Pharmacist,Physical Therapist,Radiologic Technologist,X-ray Technologist,Grand Total
0,REGION I (ILOCOS REGION),PH010000000,ILOCOS NORTE,PH012800000,ADAMS,PH012801000,Public,0,1,0,3,4,0,0,0,0,0,0,8
1,REGION I (ILOCOS REGION),PH010000000,ILOCOS NORTE,PH012800000,BACARRA,PH012802000,Private,4,5,1,6,20,0,0,6,0,0,1,43
2,REGION I (ILOCOS REGION),PH010000000,ILOCOS NORTE,PH012800000,BACARRA,PH012802000,Public,2,1,1,10,3,0,0,0,0,0,0,17
3,REGION I (ILOCOS REGION),PH010000000,ILOCOS NORTE,PH012800000,BADOC,PH012803000,Private,0,0,0,0,0,0,0,8,0,0,0,8
4,REGION I (ILOCOS REGION),PH010000000,ILOCOS NORTE,PH012800000,BADOC,PH012803000,Public,0,1,0,8,6,0,0,0,0,0,0,15


In [6]:
df_cities = dfHW.groupby('City/Municipality code')['Grand Total'].sum()
df_cities = pd.DataFrame(df_cities.reset_index(), columns=['City/Municipality code', 'Grand Total'])
print(df_cities)

     City/Municipality code  Grand Total
0               PH012801000            8
1               PH012802000           60
2               PH012803000           23
3               PH012804000           51
4               PH012805000          414
...                     ...          ...
1347            PH175913000            2
1348            PH175914000           17
1349            PH175915000            8
1350            PH175917000            5
1351            Unspecified         8247

[1352 rows x 2 columns]


In [7]:
df_cities['City/Municipality code'].dropna(inplace=True)

In [8]:
df_ratio = df_active_cases.merge(df_cities, how='inner', left_on='CityMuniPSGC', right_on='City/Municipality code')
df_ratio['Grand Total'].fillna(0, inplace=True)
df_ratio

,CityMuniPSGC,active_cases,City/Municipality code,Grand Total
0,PH137404000,2366,PH137404000,9568
1,PH137403000,856,PH137403000,1674
2,PH137604000,726,PH137604000,1432
3,PH137607000,682,PH137607000,859
4,PH137602000,674,PH137602000,3239
...,...,...,...,...
1081,PH086007000,1,PH086007000,1
1082,PH086004000,1,PH086004000,9
1083,PH084822000,1,PH084822000,6
1084,PH084814000,1,PH084814000,12


In [9]:
df_ratio['ratio'] = df_ratio['Grand Total'] / df_ratio['active_cases']

In [10]:
# df_ratio.to_csv('ratio.csv')

In [11]:
df_ratio[df_ratio['ratio'] < 1].head()

,CityMuniPSGC,active_cases,City/Municipality code,Grand Total,ratio
12,PH045805000,331,PH045805000,55,0.166163
13,PH045802000,318,PH045802000,203,0.638365
16,PH042103000,270,PH042103000,194,0.718519
22,PH031420000,191,PH031420000,179,0.937173
39,PH031411000,101,PH031411000,17,0.168317


In [18]:
gdf_ratio = gpd.GeoDataFrame(df_ratio)

In [17]:
gdf = gpd.read_file('./City-Municipality/phl_admbnda_adm3_psa_namria_20200529.shp')
gdf.head()

,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,1.601219,0.063496,Aborlan,PH175301000,None,None,None,Palawan,PH175300000,Region IV-B,PH170000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((118.58350 9.37700, 118.58398 9..."
1,1.078749,0.050232,Abra de Ilog,PH175101000,None,None,None,Occidental Mindoro,PH175100000,Region IV-B,PH170000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.58412 13.50198, 120.58420 13.501..."
2,0.424301,0.006453,Abucay,PH030801000,None,None,None,Bataan,PH030800000,Region III,PH030000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.49873 14.75614, 120.49891 14.755..."
3,0.566053,0.011343,Abulug,PH021501000,None,None,None,Cagayan,PH021500000,Region II,PH020000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.43455 18.46651, 121.43502 18.466..."
4,1.013649,0.026124,Abuyog,PH083701000,None,None,None,Leyte,PH083700000,Region VIII,PH080000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.02684 10.73500, 125.02683 ..."


In [20]:
gdf_ratio_merged = gpd.GeoDataFrame.merge(gdf_ratio, gdf, how='inner', left_on='CityMuniPSGC', right_on='ADM3_PCODE')
gdf_ratio_merged.set_crs('epsg:4326')

c:\Users\baja0\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\geodataframe.py:1503: FutureWarning: Accessing CRS of a GeoDataFrame without a geometry column is deprecated and will be removed in GeoPandas 0.12. Use GeoDataFrame.set_geometry to set the active geometry column.
  result.crs = self.crs


,CityMuniPSGC,active_cases,City/Municipality code,Grand Total,ratio,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,ADM3_REF,...,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,PH137404000,2366,PH137404000,9568,4.043956,0.773902,0.013630,Quezon City,PH137404000,None,...,"NCR, Second District",PH137400000,National Capital Region,PH130000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.13497 14.77673, 121.13447 14.776..."
1,PH137403000,856,PH137403000,1674,1.955607,0.344848,0.002626,City of Pasig,PH137403000,None,...,"NCR, Second District",PH137400000,National Capital Region,PH130000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.08469 14.62389, 121.08579 14.622..."
2,PH137604000,726,PH137604000,1432,1.972452,0.431594,0.003758,City of Parañaque,PH137604000,City of Paranaque,...,"NCR, Fourth District",PH137600000,National Capital Region,PH130000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((120.99814 14.53308, 120.99883 14.532..."
3,PH137607000,682,PH137607000,859,1.259531,0.358176,0.002988,Taguig City,PH137607000,None,...,"NCR, Fourth District",PH137600000,National Capital Region,PH130000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.08105 14.54749, 121.08109 14.547..."
4,PH137602000,674,PH137602000,3239,4.805638,0.260317,0.002094,City of Makati,PH137602000,None,...,"NCR, Fourth District",PH137600000,National Capital Region,PH130000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((121.03468 14.56725, 121.03521 14.567..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,PH086007000,1,PH086007000,1,1.000000,1.181987,0.025291,Gandara,PH086007000,None,...,Samar,PH086000000,Region VIII,PH080000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((124.67039 11.98611, 124.67060 ..."
1082,PH086004000,1,PH086004000,9,9.000000,0.610236,0.016242,Calbiga,PH086004000,None,...,Samar,PH086000000,Region VIII,PH080000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((124.98327 11.68170, 124.99279 11.677..."
1083,PH084822000,1,PH084822000,6,6.000000,0.695723,0.021251,Silvino Lobos,PH084822000,None,...,Northern Samar,PH084800000,Region VIII,PH080000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"POLYGON ((124.86184 12.14976, 124.85798 12.137..."
1084,PH084814000,1,PH084814000,12,12.000000,0.841933,0.014823,Palapag,PH084814000,None,...,Northern Samar,PH084800000,Region VIII,PH080000000,Philippines (the),PH,2016-06-30,2020-05-29,None,"MULTIPOLYGON (((125.03200 12.45800, 125.01206 ..."


In [21]:
gdf_ratio_merged.to_file('/Users/baja0/OneDrive/Desktop/Data Camp/health_workers_active_case_ratio/health_workers_active_case_ratio.shp')

C:\Users\baja0\AppData\Local\Temp\ipykernel_11232\2520208746.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_ratio_merged.to_file('/Users/baja0/OneDrive/Desktop/Data Camp/health_workers_active_case_ratio/health_workers_active_case_ratio.shp')
